In [9]:
%env NUMBA_CUDA_LOW_OCCUPANCY_WARNINGS=0
import torch
import numba.cuda as cu
import matplotlib.pyplot as plt
from flood import rightflood

env: NUMBA_CUDA_LOW_OCCUPANCY_WARNINGS=0


# 不规则数组的定义
如果数组$A$为多维数组，且某个维度上的子数组长度不一致，则称它为**不规则数组**。

例如以下二维数组为不规则数组，因为它的第0维上的子数组长度不一致，分别为$3,1,2$：

$$
\left[
\begin{aligned}
	&[ 6, 5, 5    &] \\
	&[ 2          &] \\
	&[ 9, 9       &] \\
\end{aligned}
\right]
$$

在使用规则数组（张量）来表示不规则数组时，通常会采用以下形式，例如对不规则数组$C$有：

$$
C = \begin{pmatrix}
	D^C & L^C
\end{pmatrix}
$$

其中$D^C$为数据数组，$L^C$为索引数组。

假设索引从0开始，它们按照如下方式定义了不规则数组$C$：

$$
C_{ij} = D^C_{L^C_i + j} \\
j < L^C_{i+1} - L^C_i
$$

设$C$的第0维的长度（子数组的数量）为$n^C$，
则$L^C$为长度为$n^C+1$的1维向量，
而$D^C$的第0维的长度为$L^C_{n^C}$。

例如对上述的例子而言，有：

$$
\begin{aligned}

D^C &= [6,5,5,2,9,9] \\ 
L^C &= [0,    3,4,6] \\

\end{aligned}
$$

# 二维不规则数组的遍历
本节中将省去上标$^C$以简化记号。

设有二维不规则数组$C=(D, L)$：

In [2]:
D = torch.tensor([6,5,5,2,9,9])
L = torch.tensor([0,3,4,6])

print(f"D = {D.cpu().numpy()}")
print(f"L = {L.cpu().numpy()}")

D = [6 5 5 2 9 9]
L = [0 3 4 6]


我们现在想遍历$C_{ij}$，要求处理每个$C_{ij}$的时候都能知道其对应的$i$和$j$。

那么显然要遍历$C_{ij}$，其实就是遍历$D_k$：

``` python
for i in range(n):
  for k in range(L[i], L[i+1]):
    j = k - L[i]
    Process D[k] == C[i][j]
```

不过这种遍历方法的内层循环`for k in range(L[i], L[i+1])`是一个不定长循环（指定不同`i`时迭代次数不同），
所以如果我开`n`个线程并行执行外层循环的话，由于内层循环的迭代次数不同，所以线程之间没法同步执行，从而也就不适合GPU实现。

适合GPU的理想的遍历方法应该是：

``` python
for k in range(0, L[n]):
  Decompose k into i, j
  Process D[k] == C[i][j]
```

这里的问题是如何把`k`分解为`i`和`j`。

我的解决方案是引入两个和$D$等长的辅助数组用于计算$i, j$：
* $I$：$i = I_k$
* $S$：$j = k - S_k$

首先来看$I$的计算，它分为两步：

1. 在$L_i$对应的位置填入$i$，即$I_{L_i} \leftarrow i, i \in [0, n)$。
2. 对$I$进行[向右洪溢](https://zhuanlan.zhihu.com/p/583315932)，将上一步填入的数值扩散进整个数组中。

In [8]:
I = torch.zeros_like(D)
print(f"0. I = {I.cpu().numpy()}")

I.index_add_(dim=0, index=L[:-1], source=torch.arange(start=0, end=L.shape[0]-1), alpha=1)
print(f"1. I = {I.cpu().numpy()}")

I = rightflood(I)
print(f"2. I = {I.cpu().numpy()}")

0. I = [0 0 0 0 0 0]
1. I = [0 0 0 1 2 0]
2. I = [0 0 0 1 2 2]


$S$的计算也是类似的两步：

1. 在$L_i$对应的位置填入$L_i$，即$S_{L_i} \leftarrow L_i, i \in [0, n)$。
2. 对$S$进行[向右洪溢](https://zhuanlan.zhihu.com/p/583315932)，将上一步填入的数值扩散进整个数组中。

In [12]:
S = torch.zeros_like(D)
print(f"0. S = {S.cpu().numpy()}")

S.index_add_(dim=0, index=L[:-1], source=L[:-1], alpha=1)
print(f"1. S = {S.cpu().numpy()}")

S = rightflood(S)
print(f"2. S = {S.cpu().numpy()}")

0. S = [0 0 0 0 0 0]
1. S = [0 0 0 3 4 0]
2. S = [0 0 0 3 4 4]


于是可以对不规则数组$C$按如下方式进行遍历：

``` python
for k in range(0, L[n]):
  i = I[k]
  j = k - S[k]
  Process D[k] == C[i][j]
```

例如如果我要将$C$中每个元素都设为$|i-j|$，即$C_{ij} \leftarrow |i-j|$：

In [16]:
# Convenient function for CUDA
block_size=512
def cal_block_num(n):
	return int((n-1)/block_size) + 1

In [18]:
# Move to GPU
D = D.to(device="cuda")
L = L.to(device="cuda")
I = I.to(device="cuda")
S = S.to(device="cuda")

In [20]:
# Kernel Function
@cu.jit
def traverse(D, L, I, S):
	k = cu.grid(1)
	if k >= D.shape[0]:
		return
	i = I[k]
	j = k - S[k]
	D[k] = abs(i-j)

traverse[cal_block_num(D.shape[0]), block_size](D, L, I, S)
print(f"D = {D.cpu().numpy()}")

D = [0 1 2 1 2 1]


将其打印成二维不规则数组的样子：

In [27]:
print("C = [")
k = 0
for i in range(L.shape[0]-1):
	print("  [", end=" ")
	for j in range(L[i], L[i+1]):
		print(D[k].cpu().numpy(), end=" ")
		k += 1
	print("]")
print("]")

C = [
  [ 0 1 2 ]
  [ 1 ]
  [ 2 1 ]
]


# 三维不规则数组的遍历
如果一个不规则数组$C=(D^C, L^C)$在第1维上的每个元素$C_{ij}$都是长度不一致的一维数组的话，
那么它就是一个三维不规则数组。

例如一个三维不规则数组：

$$
\begin{pmatrix}
	\begin{pmatrix}
		C_{00} = 
		\begin{pmatrix}
			1 & 2 & 3
		\end{pmatrix}
		\\
		C_{01} = 
		\begin{pmatrix}
			4 & 5
		\end{pmatrix}
		\\
	\end{pmatrix}
	\\
	\begin{pmatrix}
		C_{10} = 
		\begin{pmatrix}
			6 & 7 & 8 & 9
		\end{pmatrix}
		\\
		C_{11} = 
		\begin{pmatrix}
			10
		\end{pmatrix}
		\\
		C_{12} = 
		\begin{pmatrix}
			11 & 12
		\end{pmatrix}
		\\
		C_{13} = 
		\begin{pmatrix}
			13 & 14 & 15
		\end{pmatrix}
		\\
	\end{pmatrix}
\end{pmatrix}
$$

显然，此时$C$的第0维上的每个元素$C_i$都会是一个二维不规则数组：

$$
\begin{pmatrix}
	C_{0} = 
	\begin{pmatrix}
		\begin{pmatrix}
			1 & 2 & 3
		\end{pmatrix}
		\\
		\begin{pmatrix}
			4 & 5
		\end{pmatrix}
		\\
	\end{pmatrix}
	\\
	C_{1} = 
	\begin{pmatrix}
		\begin{pmatrix}
			6 & 7 & 8 & 9
		\end{pmatrix}
		\\
		\begin{pmatrix}
			10
		\end{pmatrix}
		\\
		\begin{pmatrix}
			11 & 12
		\end{pmatrix}
		\\
		\begin{pmatrix}
			13 & 14 & 15
		\end{pmatrix}
		\\
	\end{pmatrix}
\end{pmatrix}
$$


**TODO**